In [10]:
from langchain_community.document_loaders import PyPDFLoader

In [11]:
pdf_path = "../data/test_document.pdf"

In [12]:
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [13]:
print(f"Документ успешно загружен.")
print(f"Тип объекта, который мы получили: {type(documents)}")
print(f"Количество элементов в списке: {len(documents)}")
print(f"Тип одного элемента: {type(documents[0])}")

Документ успешно загружен.
Тип объекта, который мы получили: <class 'list'>
Количество элементов в списке: 25
Тип одного элемента: <class 'langchain_core.documents.base.Document'>


In [14]:
first_page_document = documents[0]

print("\n--- МЕТАДАННЫЕ ПЕРВОЙ СТРАНИЦЫ ---")
print(first_page_document.metadata)

print("\n--- СОДЕРЖИМОЕ (КОНТЕНТ) ПЕРВОЙ СТРАНИЦЫ ---")
print(first_page_document.page_content)


--- МЕТАДАННЫЕ ПЕРВОЙ СТРАНИЦЫ ---
{'producer': 'GPL Ghostscript 10.02.1', 'creator': 'PDF24 Creator', 'creationdate': '2025-06-23T11:47:28+03:00', 'moddate': '2025-06-23T11:47:28+03:00', 'title': '<CFEEF1F2E0EDEEE2EBE5EDE8E520CFF0E0E2E8F2E5EBFCF1F2E2E020D0D420>', 'author': 'petr.baldaev', 'source': '../data/test_document.pdf', 'total_pages': 25, 'page': 0, 'page_label': '1'}

--- СОДЕРЖИМОЕ (КОНТЕНТ) ПЕРВОЙ СТРАНИЦЫ ---
Постановление Правительства РФ от 24.12.2021 N 2464 (ред. от 12.06.2024) "О порядке обучения по охране труда и проверки знания требований охраны труда" (вместе с 
"Правилами обучения по охране труда и проверки знания требований охраны труда") 
ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ 
ПОСТАНОВЛЕНИЕ 
от 24 декабря 2021 г. N 2464 
О ПОРЯДКЕ 
ОБУЧЕНИЯ ПО ОХРАНЕ ТРУДА И ПРОВЕРКИ ЗНАНИЯ ТРЕБОВАНИЙ 
ОХРАНЫ ТРУДА 
Список изменяющих документов 
(в ред. Постановлений Правительства РФ от 30.12.2022 N 2540,
от 12.06.2024 N 792)
В соответствии с частью третьей статьи 219 Трудовог о код

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

In [17]:
chunks = text_splitter.split_documents(documents)

In [18]:
print(f"Было {len(documents)} страниц, а стало {len(chunks)} чанков.")

print("\n--- Пример первого чанка ---")
# Убедимся, что чанк существует, прежде чем выводить его
if len(chunks) > 0:
    print(chunks[0].page_content)
    print(f"\nДлина: {len(chunks[0].page_content)} символов")
    print(f"Метаданные: {chunks[0].metadata}")

print("\n--- Пример второго чанка ---")
# То же самое для второго чанка
if len(chunks) > 1:
    print(chunks[1].page_content)
    print(f"\nДлина: {len(chunks[1].page_content)} символов")
    print(f"Метаданные: {chunks[1].metadata}")

Было 25 страниц, а стало 133 чанков.

--- Пример первого чанка ---
Постановление Правительства РФ от 24.12.2021 N 2464 (ред. от 12.06.2024) "О порядке обучения по охране труда и проверки знания требований охраны труда" (вместе с 
"Правилами обучения по охране труда и проверки знания требований охраны труда") 
ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ 
ПОСТАНОВЛЕНИЕ 
от 24 декабря 2021 г. N 2464 
О ПОРЯДКЕ 
ОБУЧЕНИЯ ПО ОХРАНЕ ТРУДА И ПРОВЕРКИ ЗНАНИЯ ТРЕБОВАНИЙ 
ОХРАНЫ ТРУДА 
Список изменяющих документов 
(в ред. Постановлений Правительства РФ от 30.12.2022 N 2540,
от 12.06.2024 N 792)
В соответствии с частью третьей статьи 219 Трудовог о кодекса Российской Федерации 
Правительство Российской Федерации постановляет: 
1. Утвердить прилагаемые Правила обучения по охране т руда и проверки знания требований охраны 
труда (далее - Правила). 
2. Установить, что: 
положения пункта 78 Правил применяются с 1 марта 2023 г.; 
положения пункта 99 Правил в части, касающейся осущес твления работодателем деят

In [19]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [20]:
load_dotenv()

# Проверяем, что ключ успешно загрузился
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Не найден OPENAI_API_KEY. Убедитесь, что он есть в файле .env")

In [21]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [22]:
# Указываем папку, куда будет сохранена база данных
persist_directory = 'chroma_db'

# Создаем базу данных
db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model,
    persist_directory=persist_directory
)

In [23]:
print("База данных успешно создана и сохранена.")
print(f"В ней содержится {db._collection.count()} документов.")

База данных успешно создана и сохранена.
В ней содержится 266 документов.


In [24]:
# Указываем ту же папку, что и при сохранении
persist_directory = 'chroma_db'

# Инициализируем модель эмбеддингов (она нужна для векторизации нашего вопроса)
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Загружаем базу данных с диска
db = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings_model
)

print("База данных успешно загружена с диска.")

База данных успешно загружена с диска.


In [25]:
query = "Каковы виды обучения по охране труда бывают?"

In [26]:
relevant_docs = db.similarity_search(query, k=3)

In [27]:
print(f"Найдено {len(relevant_docs)} релевантных чанков для вопроса: '{query}'\n")
print("-" * 50)

# Распечатаем содержимое и метаданные каждого найденного чанка
for i, doc in enumerate(relevant_docs):
    print(f"РЕЗУЛЬТАТ #{i+1}\n")
    print(f"Источник: {doc.metadata.get('source', 'Неизвестно')}, Страница: {doc.metadata.get('page', 'Неизвестно')}\n")
    print("Содержимое:")
    print(doc.page_content)
    print("\n" + "-" * 50)

Найдено 3 релевантных чанков для вопроса: 'Каковы виды обучения по охране труда бывают?'

--------------------------------------------------
РЕЗУЛЬТАТ #1

Источник: ../data/test_document.pdf, Страница: 1

Содержимое:
специализированным процессом получения знаний, умений и навыков. 
4. Обучение по охране труда осуществляется в ходе проведения: 
а) инструктажей по охране труда; 
б) стажировки на рабочем месте; 
в) обучения по оказанию первой помощи пострадавшим; 
г) обучения по использованию (применению) средств индивидуальной защиты; 
д) обучения по охране труда у работодателя, в том чис ле обучения безопасным методам и 
приемам 
выполнения работ, или в организации, у индивидуального п редпринимателя, оказывающих услуги по 
проведению обучения по охране труда (далее - обучение требованиям охраны труда). 
5. Настоящие Правила не распространяются на обучение по  охране труда и проверку знания 
требований охраны труда, предусмотренные специальными тр ебованиями к проведению обучения по 
ох

In [28]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [29]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [30]:
template = """
Используй следующие фрагменты контекста, чтобы ответить на вопрос в конце.
Если ты не знаешь ответа, просто скажи, что не знаешь, не пытайся придумать ответ.
Ответ должен быть максимально лаконичным и по делу. Предоставляй ответ только на русском языке.

Контекст: {context}

Вопрос: {question}

Полезный ответ:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [34]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Наш поисковик из ChromaDB
retriever = db.as_retriever(search_kwargs={"k": 3})

# Создаем цепочку
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | QA_CHAIN_PROMPT
    | llm
    | StrOutputParser()
)

In [40]:
# Используем тот же вопрос
query = "Каковы виды обучения по охране труда бывают?"

# получаем релевантные документы для вопроса
retrieved_docs = retriever.invoke(query)


# Запускаем всю цепочку
final_answer = rag_chain.invoke(query)

# Печатаем результат
print("ФИНАЛЬНЫЙ ОТВЕТ:")
print(final_answer)

print("\n" + "="*50 + "\n")

print("ИСПОЛЬЗОВАННЫЕ ИСТОЧНИКИ:")
for doc in retrieved_docs:
    print(f"- Источник: {doc.metadata.get('source', 'N/A')}, Страница: {doc.metadata.get('page', 'N/A')}")

ФИНАЛЬНЫЙ ОТВЕТ:
Инструктажи, стажировка на рабочем месте, обучение по оказанию первой помощи пострадавшим, обучение по использованию средств индивидуальной защиты, обучение по охране труда у работодателя или организации.


ИСПОЛЬЗОВАННЫЕ ИСТОЧНИКИ:
- Источник: ../data/test_document.pdf, Страница: 1
- Источник: ../data/test_document.pdf, Страница: 1
- Источник: ../data/test_document.pdf, Страница: 7
